In [ ]:
import json

def compute_required_skill_similarity(candidate_skill, job_req_skill_value):
    """
    Given a candidate skill (which may be a string or a list of strings) and a 
    job-required skill value (a nested list of terms), compute a similarity score.
    
    For each inner list (group) in job_req_skill_value:
      - For each candidate term, compute the similarity with every term in the group
        and take the maximum similarity.
      - Average these maximum similarities (one per candidate term) to get a group score.
    
    If there's only one group, return its score; if multiple groups, return the maximum group score.
    """
    # Normalize candidate_skill to a list of strings.
    if isinstance(candidate_skill, str):
        candidate_terms = [candidate_skill]
    elif isinstance(candidate_skill, list):
        candidate_terms = candidate_skill
    else:
        candidate_terms = []

    if not job_req_skill_value or not candidate_terms:
        return 0.0

    inner_avgs = []
    # Process each group in the job requirement.
    for group in job_req_skill_value:
        if not group:
            continue
        candidate_max_sims = []
        for cand_term in candidate_terms:
            sims = []
            for req_term in group:
                sim = nlp_similarity_cached(cand_term, req_term).item()
                sims.append(sim)
            max_sim = max(sims) if sims else 0.0
            candidate_max_sims.append(max_sim)
        group_avg = sum(candidate_max_sims) / len(candidate_max_sims) if candidate_max_sims else 0.0
        inner_avgs.append(group_avg)
    best_avg = max(inner_avgs) if inner_avgs else 0.0
    return best_avg

def calculate_skill_match_score(job_description_json, resume_json, skill_type='mandatory'):
    """
    Calculates an overall match score for skills (mandatory or preferred) and returns a JSON.
    
    For each job-required skill:
      - For each candidate skill (if candidate's years >= minyears), compute the similarity
        between the candidate's skill and the job requirement (using compute_required_skill_similarity).
      - Record the candidate skill with the highest similarity for that job requirement.
    
    Returns a JSON string containing:
      - overall_score: The average of the best-match scores for each job-required skill.
      - details: A list of objects, each with the job requirement, minimum years required,
                 the candidate skill that matched best, and the associated similarity score.
    """
    if skill_type == 'mandatory':
        job_skills = extract_job_mandatory_skills(job_description_json)
    else:
        job_skills = extract_job_preferred_skills(job_description_json)
        
    resume_skills = extract_resume_skills(resume_json)
    requirement_details = []
    requirement_scores = []
    
    for req in job_skills:
        job_req_skill_value = req.get('skill', [])
        min_years_required = req.get('minyears', [0])[0]
        
        best_match = 0.0
        best_candidate_skill = None
        
        # Iterate over each candidate skill entry.
        for candidate in resume_skills:
            candidate_years = candidate.get('years', 0)
            if candidate_years >= min_years_required:
                for candidate_skill in candidate.get('skill', []):
                    sim = compute_required_skill_similarity(candidate_skill, job_req_skill_value)
                    if sim > best_match:
                        best_match = sim
                        best_candidate_skill = candidate_skill
        requirement_details.append({
            "job_requirement": job_req_skill_value,
            "min_years_required": min_years_required,
            "matched_candidate_skill": best_candidate_skill,
            "similarity_score": best_match
        })
        requirement_scores.append(best_match)
    
    overall_score = sum(requirement_scores) / len(requirement_scores) if requirement_scores else 0.0
    result = {"overall_score": overall_score, "details": requirement_details}
    
    return json.dumps(result, indent=2)